In [1]:
# import required packages
import pandas as pd
from pulp import *
import numpy as np

In [2]:
# import and prepare data
diet0 = pd.read_excel('diet.xls', sheet_name = 'Sheet1')
diet = diet0[:64]
diet.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [3]:
diet.columns

Index(['Foods', 'Price/ Serving', 'Serving Size', 'Calories', 'Cholesterol mg',
       'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g',
       'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg'],
      dtype='object')

In [4]:
daily_intake = diet0[-2:]
daily_intake = daily_intake.drop(columns=['Foods', 'Price/ Serving', 'Serving Size'])
daily_intake

,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [5]:
food = diet.Foods.tolist()
all_data = {}
for j in ['Price/ Serving', 'Calories', 'Cholesterol mg',
       'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g',
       'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg']:
    all_data[j] = dict([(food[i], diet[j][i]) for i in range(len(food))])
all_data.keys()

dict_keys(['Price/ Serving', 'Calories', 'Cholesterol mg', 'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g', 'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg'])

In [6]:
# PuLP create the problem
# this is a minimize problem - minimizing cost
prob = LpProblem('Minimizing_Diet_Cost', LpMinimize)

In [7]:
# create variables to be optimized, variables must be 0 or bigger
variables = LpVariable.dicts('type', food, 0)

In [8]:
# add objective function
prob += lpSum([variables[i]*all_data['Price/ Serving'][i] for i in food]), 'cost_of_food'

In [9]:
# add constraints, minimal and maximal
for k in ['Calories', 'Cholesterol mg', \
          'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g',\
          'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg']:
    prob += lpSum([variables[i]*all_data[k][i] for i in food]) >= daily_intake[k].tolist()[0], 'min_'+k
    prob += lpSum([variables[i]*all_data[k][i] for i in food]) <= daily_intake[k].tolist()[1], 'max_'+k

In [10]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [11]:
# Final result for the optimization

print('Food selection to minimize cost:\n')
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue, 'servings')
print("\nTotal Diet Cost per Day = $", round(value(prob.objective), 2))

Food selection to minimize cost:

type_Celery,_Raw = 52.64371 servings
type_Frozen_Broccoli = 0.25960653 servings
type_Lettuce,Iceberg,Raw = 63.988506 servings
type_Oranges = 2.2929389 servings
type_Poached_Eggs = 0.14184397 servings
type_Popcorn,Air_Popped = 13.869322 servings

Total Diet Cost per Day = $ 4.34


In [12]:
# additional constraints should be added to make the diet more delicious :-) 

Adding additional constraints to the problem:
a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)

b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

In [13]:
# create the problem
prob1 = LpProblem('Minimizing_Diet_Cost', LpMinimize)

# add objective function
prob1 += lpSum([variables[i]*all_data['Price/ Serving'][i] for i in food]), 'cost_of_food'

In [14]:
# Condition a
# If a food is selected, then a minimum of 1/10 serving must be chosen
# create additional variables to decide if the food type is chosen
# this variable is binary with values 0 (not chosen) and 1 (chosen)

y = LpVariable.dicts('choose', food, 0, 1, LpInteger)

# add the minimum 0.1 serving if chosen constraint to the problem
for i in food:
    prob1 += variables[i] >= y[i]*0.1
    # originally I only add the above condition variables[i] >= y[i]*0.1
    # turn out another condition is needed to prevent y to be zero
    # added as follows
    prob1 += variables[i] <= y[i]*1000000

In [15]:
# Condition b
# celery and frozen broccoli, at most 1

celery_broccoli = ['Frozen Broccoli', 'Celery, Raw']

prob1 += lpSum([y[i] for i in celery_broccoli]) <= 1, 'at most 1 celery and frozen broccoli'

In [16]:
# Condition c
# define a protein list
protein = [ 'Tofu', 'Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey',
 'Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', 'Hotdog, Plain', 'Pork',
 'Sardines in Oil', 'White Tuna in Water']

prob1 += lpSum([y[i] for i in protein]) >= 3, 'at least 3 proteins'

In [17]:
# add minimal and maximal daily intake constraints
for k in ['Calories', 'Cholesterol mg', \
          'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g',\
          'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg']:
    prob1 += lpSum([variables[i]*all_data[k][i] for i in food]) >= daily_intake[k].tolist()[0], 'min_'+k
    prob1 += lpSum([variables[i]*all_data[k][i] for i in food]) <= daily_intake[k].tolist()[1], 'max_'+k

In [18]:
prob1.solve()
print("Status:", LpStatus[prob1.status])

Status: Optimal


In [19]:
# Final result for the optimization

print('Food selection to minimize cost with 3 additional constraints:\n')
for v in prob1.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue, 'servings')
print("\nTotal Diet Cost per Day = $", round(value(prob1.objective), 2))

Food selection to minimize cost with 3 additional constraints:

choose_Celery,_Raw = 1.0 servings
choose_Kielbasa,Prk = 1.0 servings
choose_Lettuce,Iceberg,Raw = 1.0 servings
choose_Oranges = 1.0 servings
choose_Peanut_Butter = 1.0 servings
choose_Poached_Eggs = 1.0 servings
choose_Popcorn,Air_Popped = 1.0 servings
choose_Scrambled_Eggs = 1.0 servings
type_Celery,_Raw = 42.399358 servings
type_Kielbasa,Prk = 0.1 servings
type_Lettuce,Iceberg,Raw = 82.802586 servings
type_Oranges = 3.0771841 servings
type_Peanut_Butter = 1.9429716 servings
type_Poached_Eggs = 0.1 servings
type_Popcorn,Air_Popped = 13.223294 servings
type_Scrambled_Eggs = 0.1 servings

Total Diet Cost per Day = $ 4.51
